# Projeto de Ranking de políticos brasileiros V3

Esse projeto foi sugerido no linkedin.com com o tema política.
Existem pelo menos dois rankings de políticos disponíveis na internet.

O Legisla Brasil: https://lnkd.in/ecRdMvbV

e o Ranking dos Políticos: https://lnkd.in/eubz2hwK

O ponto principal é descobrir se existe grau de associação entre eles. "Ou seja, um candidato é bom e ruim a depender do ranking? Se sim, por que?"

Checkpoints sugeridos:
1) Coleta de dados via web scrapping. Bibliotecas como Selenium e BeautifulSoup podem ajudar. <br>
2) Análise exploratória: Gráficos de dispersão entre nota padronizada dos dois rankings. São correlacionados? Existem outliers?<br>
3) Modelo explicativo: Regressão linear múltipla entre os 4 itens do ranking do Legisla Brasil e a nota do Ranking dos Políticos, por exemplo. Os 4 itens do Legisla explicam qual variabilidade do Ranking dos Políticos?<br>
4) Discussão: Por que são diferentes? Por que são iguais? Existem viéses políticos? São apartidários? Qual sua opinião que justifique os resultados?<br>

In [1]:
# Importação das bibliotecas 
from bs4 import BeautifulSoup
import requests 
import pandas as pd
import numpy as np
import re
from unidecode import unidecode #retirar acentos
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from time import sleep
from unidecode import unidecode

## Lista do site Legisla Brasil: https://indice.legislabrasil.org/public/explorador

In [2]:
# formatando a janela do chrome que irá ser usada na pesquisa do selenium
options = Options()
#options.add_argument('--headless') #utilizar quando não desejar que a página abra
options.add_argument('window-size=1000,800')

In [3]:
# configurando o webdriver e abrindo o navegador
navegador = webdriver.Chrome(options=options)

navegador.get('https://indice.legislabrasil.org/public/explorador')
sleep(1)
# para pegar o conteudo da página
pag1_source = BeautifulSoup(navegador.page_source)
contador = pag1_source.find_all('div', {'class', 'box_dep'} )
num_dep = len(list(contador))

In [4]:
lista_geral = []
#entrando na página de notas de cada candidato
for n in range(1,num_dep+1):

    bc = '//*[@id="dados"]/div/div[4]/div['+str(n)+']/a/img'
    click_dep = navegador.find_element(By.XPATH, bc)
    click_dep.click()

    #pegando o html individual de cada candidato
    bs_indv_cand = BeautifulSoup(navegador.page_source)
    #procurando elementos h2 no html
    nome_cand = bs_indv_cand.find_all('h2')

    # 1. Nome/Estado/Partido/Notas
    #nome
    lista_informacoes = []
    lista_informacoes.append([unidecode(nome_cand[1].text.lower())])
    #estado/partido
    dados_cand = bs_indv_cand.find_all('h4')
    #formatando as saídas
    text_temp_1 = dados_cand[0].text.replace('  ','')
    #condicional referente a um valor que foge ao padrão do site
    if n == 560:
        compile_temp = re.compile('\n\n\n.*')
        text_temp2 = compile_temp.findall(text_temp_1)
        lista_informacoes[0].extend(text_temp_1.replace(text_temp2[0],'')\
                                    .replace(text_temp2[0],'').replace('\n','').split('-'))
    else:
        compile_temp = re.compile('\n\n.*')
        text_temp2 = compile_temp.findall(text_temp_1)
        lista_informacoes[0].extend(text_temp_1.replace(text_temp2[0],'')\
                                    .replace(text_temp2[1],'').replace('\n','').split('-'))

    lista_informacoes[0][1] = lista_informacoes[0][1].strip()
    lista_informacoes[0][2] = lista_informacoes[0][2].strip()
    #adicionando às notas a lista
    notas_cand = bs_indv_cand.find_all('strong', {'class': 'cor1'})
    for n in notas_cand:
         lista_informacoes[0].append(n.text)

    lista_geral.extend(lista_informacoes)

    navegador.back()

In [5]:
#transformando em dataframe
columns_name = ['nome', 'estado', 'partido', 'n_prod_leg', 'n_fisc', 'n_mobi', 'n_alin_partd']
columns_notas = ['n_prod_leg', 'n_fisc', 'n_mobi', 'n_alin_partd']
df = pd.DataFrame(lista_geral, columns=columns_name)
df[columns_notas] = df[columns_notas].apply(pd.to_numeric)
#contruindo dataframe somente com as notas
df2 = df[columns_notas]
descricao_notas = df2.apply(pd.Series.describe, axis=1)
descricao_notas

,count,mean,std,min,25%,50%,75%,max
0,4.0,2.925,4.174825,0.0,0.675,1.30,3.550,9.1
1,4.0,3.375,2.338625,0.9,2.025,3.10,4.450,6.4
2,4.0,2.825,2.597916,0.2,1.625,2.35,3.550,6.4
3,4.0,2.900,1.916594,0.9,2.025,2.60,3.475,5.5
4,4.0,6.650,2.930870,2.9,5.525,6.85,7.975,10.0
...,...,...,...,...,...,...,...,...
561,4.0,4.700,2.549510,2.0,2.825,4.75,6.625,7.3
562,4.0,4.525,2.613267,1.1,3.425,4.85,5.950,7.3
563,4.0,3.700,2.442676,0.5,2.825,3.95,4.825,6.4
564,4.0,4.625,2.043486,1.8,3.825,5.15,5.950,6.4


In [9]:
#salvando em csv
df.to_csv('rank_legislabr_completo.csv')
df2.to_csv('rank_legislabr_notas.csv')
descricao_notas.to_csv('rank_legislabr_describe.csv')